In [129]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import sys
import time

import preprocessing
import models
import prediction

from sklearn.model_selection import TimeSeriesSplit

In [16]:
path_data = '/Users/sergigomezpalleja/Downloads/'

In [216]:
train = pd.read_csv(path_data + 'raw_data_master.csv')

In [219]:
train.loc[(train.sales2.isnull()) & (train.date < '2018-01-01')].shape[0] / train.loc[train.date < '2018-01-01'].shape[0]


0.08868243243243243

In [221]:
train.loc[(~train.sales2.isnull()) & (train.date < '2018-01-01') & (train.sales2 == 0.0)]

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id
169,Cluster 1,Brand Group 12,Country 12,2012-02-01,0.0,0.0,-4.506785,-0.831747,NaN,-0.002598,NaN,NaN,170
174,Cluster 1,Brand Group 12,Country 12,2012-07-01,0.0,0.0,-3.717820,0.300827,NaN,0.000000,NaN,NaN,175
176,Cluster 1,Brand Group 12,Country 12,2012-09-01,0.0,0.0,-4.016025,-0.191202,NaN,0.000000,NaN,NaN,177
178,Cluster 1,Brand Group 12,Country 12,2012-11-01,0.0,0.0,-5.030140,-11.987181,NaN,0.012329,NaN,NaN,179
336,Cluster 1,Brand Group 12,Country 2,2012-01-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,337
337,Cluster 1,Brand Group 12,Country 2,2012-02-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,338
338,Cluster 1,Brand Group 12,Country 2,2012-03-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,339
339,Cluster 1,Brand Group 12,Country 2,2012-04-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,340
340,Cluster 1,Brand Group 12,Country 2,2012-05-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,341
341,Cluster 1,Brand Group 12,Country 2,2012-06-01,0.0,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,342


In [40]:
train.shape

(99456, 13)

In [41]:
list(train)

['Cluster',
 'Brand_Group',
 'Country',
 'date',
 'sales1',
 'sales2',
 'inv1',
 'inv2',
 'inv3',
 'inv4',
 'inv5',
 'inv6',
 'id']

In [42]:
train.head(n=1)

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id
0,Cluster 1,Brand Group 12,Country 10,2012-01-01,44.214973,41.311052,NaN,NaN,0.0,0.0,NaN,NaN,1


In [52]:
df = train.copy()

In [53]:
df['Country'] = df['Country'].str.extract('(\d+)').astype(int)
df['Brand_Group'] = df['Brand_Group'].str.extract('(\d+)').astype(int)
df['Cluster'] = df['Cluster'].str.extract('(\d+)').astype(int)

/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


In [54]:
df.head()

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id
0,1,12,10,2012-01-01,44.214973,41.311052,NaN,NaN,0.000000,0.000000,NaN,NaN,1
1,1,12,10,2012-02-01,104.918692,95.697737,NaN,NaN,-3.441472,0.000000,NaN,NaN,2
2,1,12,10,2012-03-01,25.813126,21.010914,NaN,NaN,0.000000,0.000000,NaN,NaN,3
3,1,12,10,2012-04-01,21.735670,19.227058,NaN,NaN,0.000000,0.000000,NaN,NaN,4
4,1,12,10,2012-05-01,17.258679,15.390803,NaN,NaN,0.000000,-0.010893,NaN,NaN,5


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99456 entries, 0 to 99455
Data columns (total 13 columns):
Cluster        99456 non-null int64
Brand_Group    99456 non-null int64
Country        99456 non-null int64
date           99456 non-null object
sales1         74520 non-null float64
sales2         77688 non-null float64
inv1           40236 non-null float64
inv2           40488 non-null float64
inv3           13440 non-null float64
inv4           50148 non-null float64
inv5           420 non-null float64
inv6           32844 non-null float64
id             99456 non-null int64
dtypes: float64(8), int64(4), object(1)
memory usage: 9.9+ MB


In [113]:
train = pd.read_csv(path_data + 'train_preprocessed_sergi_1.csv')

In [114]:
df = train.copy()

In [115]:
df['date'] = df['date'].astype('datetime64[ns]')

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99456 entries, 0 to 99455
Data columns (total 13 columns):
Cluster        99456 non-null int64
Brand_Group    99456 non-null int64
Country        99456 non-null int64
date           99456 non-null datetime64[ns]
sales1         74520 non-null float64
sales2         77688 non-null float64
inv1           40236 non-null float64
inv2           40488 non-null float64
inv3           13440 non-null float64
inv4           50148 non-null float64
inv5           420 non-null float64
inv6           32844 non-null float64
id             99456 non-null int64
dtypes: datetime64[ns](1), float64(8), int64(4)
memory usage: 9.9 MB


In [109]:
print("Percentage Missings Inv 1 ", df.loc[df.inv1.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 2 ", df.loc[df.inv2.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 3 ", df.loc[df.inv3.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 4 ", df.loc[df.inv4.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 5 ", df.loc[df.inv5.isnull()].shape[0]/df.shape[0])
print("Percentage Missings Inv 6 ", df.loc[df.inv6.isnull()].shape[0]/df.shape[0])

Percentage Missings Inv 1  0.5954391891891891
Percentage Missings Inv 2  0.5929054054054054
Percentage Missings Inv 3  0.8648648648648649
Percentage Missings Inv 4  0.49577702702702703
Percentage Missings Inv 5  0.995777027027027
Percentage Missings Inv 6  0.6697635135135135


In [119]:
df.loc[(df.inv5.isnull()) & (df.inv5 != 0)]

,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id


In [117]:
df['inv5'].fillna(value = 0.0, inplace = True)

In [177]:
df = pd.read_csv(path_data + 'train_preprocessed_sergi_1.csv')

In [178]:
df['date'] = df['date'].astype('datetime64[ns]')

In [225]:
train_X = df.loc[df.date < '2018-01-01']

In [227]:
values = train_X.date.values

In [145]:
len(values)

72

In [153]:
tscv = TimeSeriesSplit(n_splits = 5)

In [158]:
# iterating over folds, train model on each, forecast and calculate error
for train, val in tscv.split(values):
    print('Train: %s, Validation: %s' % (len(train), len(val)))
    dates_train = values[train]
    dates_val = values[val]
    #print('Train: %s, Validation: %s' % (dates_train, dates_val))

Train: 12, Validation: 12
Train: 24, Validation: 12
Train: 36, Validation: 12
Train: 48, Validation: 12
Train: 60, Validation: 12


In [171]:
dates_train

array(['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01',
       '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01',
       '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01',
       '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
       '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
       '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01',
       '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01',
       '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01',
       '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01',
       '2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
       '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
       '2015-09-01', '2015-10-01', '2015-11-01', '2015-12-01',
       '2016-01-01', '2016-02-01', '2016-03-01', '2016-04-01',
       '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01',
       '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01'],
      dtype=object)

In [179]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [182]:
X = df.loc[df.date < '2018-01-01']

In [183]:
train_X = X.loc[X.date < '2017-01-01']
val_X = X.loc[X.date >= '2017-01-01']

In [186]:
train_X.to_csv('/Users/sergigomezpalleja/Downloads/train_X.csv')
val_X.to_csv('/Users/sergigomezpalleja/Downloads/val_X.csv')

In [188]:
del train_X['date']
del train_X['id']

In [ ]:
vars_model = ['Cluster']

In [191]:
params = {

    'walk-forward-cv' : {
        "n_splits" : 5 # sets the number of folds for cross-validation
    },

    'lgb' : {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.005,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42},

    'vars_model' : ['Cluster', 'Brand_Group', 'Country', 'year', 'month', 'sales1']

}

In [206]:
df = pd.read_csv(path_data + 'pred_lasso_pablo_1.csv')

In [215]:
df

,Unnamed: 0,Cluster,Brand_Group,Country,date,sales1,sales2,inv1,inv2,inv3,inv4,inv5,inv6,id,year,month,y_hat
0,0,1,12,10,2017-01-01,12.712153,11.105362,0.000000,0.000000,0.000000,0.001951,0.0,0.000000,61,2017,1,48.713825
1,1,1,12,10,2017-02-01,13.395923,11.789671,0.000000,0.000000,0.000000,-0.010809,0.0,0.000000,62,2017,2,49.203375
2,2,1,12,10,2017-03-01,13.148651,11.512138,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,63,2017,3,49.026339
3,3,1,12,10,2017-04-01,13.236274,11.552861,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,64,2017,4,49.089073
4,4,1,12,10,2017-05-01,14.359559,12.558873,0.000000,0.000000,0.000000,-0.071319,0.0,0.000000,65,2017,5,49.893297
5,5,1,12,10,2017-06-01,17.348241,15.603540,0.000000,0.000000,-0.723368,-0.011492,0.0,0.000000,66,2017,6,52.033065
6,6,1,12,10,2017-07-01,20.395961,17.916533,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,67,2017,7,54.215101
7,7,1,12,10,2017-08-01,19.321671,17.160415,0.000000,0.000000,0.000000,-0.238904,0.0,0.000000,68,2017,8,53.445955
8,8,1,12,10,2017-09-01,18.769779,16.185335,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,69,2017,9,53.050824
9,9,1,12,10,2017-10-01,18.413925,16.467607,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,70,2017,10,52.796048


In [209]:
df_grouped = df.groupby(['Cluster', 'Brand_Group', 'month']).sum()

In [213]:
df_grouped['error'] = np.abs((df_grouped['y_hat'] - df_grouped['sales2']) / df_grouped['sales2'])

In [214]:
df_grouped

Unnamed: 0  Country        sales1       sales2  \
Cluster Brand_Group month                                                   
1       12          1            2280      519   1362.014711  1236.757869   
                    2            2300      519   1555.539798  1420.957571   
                    3            2320      519   1628.574826  1494.614561   
                    4            2340      519   1767.163271  1583.525099   
                    5            2360      519   1552.139581  1396.117515   
                    6            2380      519   1635.863903  1471.551885   
                    7            2400      519   1883.170870  1632.173591   
                    8            2420      519   1861.185972  1627.574308   
                    9            2440      519   1797.430748  1608.715411   
                    10           2460      519   1688.206774  1538.061964   
                    11           2480      519   1663.868710  1574.399619   
                    12           2500      519   1493.894214  1381.174249   
        13          1             240       46      8.426173     6.640494   
                    2             241       46     11.523210     9.402716   
                    3             242       46      7.142716     5.496543   
                    4             243       46     17.103457    14.480741   
                    5             244       46     10.630370     9.765432   
                    6             245       46     13.839012    11.746420   
                    7             246       46      8.621481     7.310123   
                    8             247       46     11.495309    10.965185   
                    9             248       46      6.780000     5.803457   
                    10            249       46      7.282222     5.608148   
                    11            250       46      8.063457     8.537778   
                    12            251       46      7.059012     6.361481   
        15          1             252       40    414.952084    88.485065   
                    2             253       40    -59.691149  -290.373364   
                    3             254       40    537.126948   613.388915   
                    4             255       40   -180.614390  -229.544362   
                    5             256       40    836.095290   798.197908   
                    6             257       40   -163.952482  -193.611296   
...                               ...      ...           ...          ...   
10      96          7           58968      164   3436.722029  3060.260216   
                    8           58974      164   3592.523464  3134.273658   
                    9           58980      164   3435.521156  3044.806001   
                    10          58986      164   3441.480548  3098.952657   
                    11          58992      164   3336.539205  2898.262451   
                    12          58998      164   3426.446871  3050.134242   
        97          1           59364      164   8845.773092  8073.877095   
                    2           59370      164   8126.245614  7424.482365   
                    3           59376      164  10136.018635  9119.044687   
                    4           59382      164   8354.465131  7522.590331   
                    5           59388      164  10212.748096  7379.530109   
                    6           59394      164   9916.688705  8645.688944   
                    7           59400      164  10202.536835  8752.329033   
                    8           59406      164   8505.099438  8454.772732   
                    9           59412      164   9163.617266  7912.554906   
                    10          59418      164   9215.499841  8071.404733   
                    11          59424      164  10328.578197  8315.121657   
                    12          59430      164   9526.545740  8296.663202   
        98          1            9936       53      0.000000     0.000000   
                    2  

In [ ]:
def prediction_absolute_prop_error(df):
    
    df_grouped = 
    return np.mean(np.abs((y_true - y_pred) / y_true))


In [285]:
all_val = pd.read_csv('/Users/sergigomezpalleja/Downloads/all_val.csv')

In [288]:
all_val.loc[all_val.sales2 == 0]

,Cluster,Brand_Group,Country,year,month,sales2,y_hat
108,1,12,34,2017,1,0.0,44.602238
109,1,12,34,2017,2,0.0,44.602238
111,1,12,34,2017,4,0.0,44.602238
112,1,12,34,2017,5,0.0,44.602238
119,1,12,34,2017,12,0.0,44.602238
120,1,12,35,2017,1,0.0,44.602238
122,1,12,35,2017,3,0.0,44.602238
123,1,12,35,2017,4,0.0,44.602238
124,1,12,35,2017,5,0.0,44.602238
125,1,12,35,2017,6,0.0,44.602238


In [267]:
df_group1 = df.loc[df['Brand_Group'].isin([51, 73, 90])]
df_group2 = df.loc[df['Brand_Group'].isin([96,97])]
df_group3 = df.loc[~df['Brand_Group'].isin([17, 24, 30, 31, 36, 41, 51, 73, 90, 96, 97])]
df_group_all = df.loc[df['Brand_Group'].isin([17, 24, 30, 31, 36, 41])]

In [268]:
df_group1['Brand_Group'] = 'Brand Group 51, 73, 90'
df_group2['Brand_Group'] = 'Brand Group 96, 97'
df_group3['Brand_Group'] = 'others'
df_group_all['Brand_Group'] = 'Brand Group ' + df_group_all['Brand_Group'].astype(str)

/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/sergigomezpalleja/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [269]:
df_group1_sum = df_group1.groupby(['Cluster', 'month', 'year', 'Brand_Group']).sum().reset_index() 
df_group2_sum = df_group2.groupby(['Cluster', 'month', 'year', 'Brand_Group']).sum().reset_index() 
df_group3_sum = df_group3.groupby(['Cluster', 'month', 'year', 'Brand_Group']).sum().reset_index() 
df_group_all_sum = df_group_all.groupby(['Cluster', 'month', 'year', 'Brand_Group']).sum().reset_index() 

In [270]:
vars_to_keep = ['Cluster', 'month', 'year', 'Brand_Group', 'sales2', 'y_hat']
df_group1_sum = df_group1_sum[vars_to_keep]
df_group2_sum = df_group2_sum[vars_to_keep]
df_group3_sum = df_group3_sum[vars_to_keep]
df_group_all_sum = df_group_all_sum[vars_to_keep]

In [275]:
df_all = pd.concat([df_group1_sum, df_group2_sum, df_group3_sum, df_group_all_sum])

In [277]:
df_all['Cluster'] = 'Cluster ' + df_all['Cluster'].astype(str)

In [281]:
df_all['error'] = np.abs(df_all['y_hat'] - df_all['sales2'])*100 / df_all['sales2']

In [283]:
individual_APE = df_all['error'].mean()

In [284]:
individual_APE

48.924871669183396

In [237]:
df_grouped = all_val.groupby(['Cluster', 'Brand_Group', 'month']).sum()

In [ ]:
brand_groups = ['17', '24', '30', '31', '36', '41']
brand_groups_51_73_90 = ['51','73', '90']
brand_groups_96_97 = ['51','73', '90']

In [238]:
df_grouped

Country   year       sales2        y_hat
Cluster Brand_Group month                                          
1       12          1          519  40340  1236.757869  9409.705884
                    2          519  40340  1420.957571  9409.705884
                    3          519  40340  1494.614561  9409.705884
                    4          519  40340  1583.525099  9409.705884
                    5          519  40340  1396.117515  9409.705884
                    6          519  40340  1471.551885  9409.705884
                    7          519  40340  1632.173591  9409.705884
                    8          519  40340  1627.574308  9409.705884
                    9          519  40340  1608.715411  9409.705884
                    10         519  40340  1538.061964  9409.705884
                    11         519  40340  1574.399619  9409.705884
                    12         519  40340  1381.174249  9409.705884
        13          1           46   2017     6.640494   470.485294
                    2           46   2017     9.402716   470.485294
                    3           46   2017     5.496543   470.485294
                    4           46   2017    14.480741   470.485294
                    5           46   2017     9.765432   470.485294
                    6           46   2017    11.746420   470.485294
                    7           46   2017     7.310123   470.485294
                    8           46   2017    10.965185   470.485294
                    9           46   2017     5.803457   470.485294
                    10          46   2017     5.608148   470.485294
                    11          46   2017     8.537778   470.485294
                    12          46   2017     6.361481   470.485294
        15          1           40   2017    88.485065   470.485294
                    2           40   2017  -290.373364   470.485294
                    3           40   2017   613.388915   470.485294
                    4           40   2017  -229.544362   470.485294
                    5           40   2017   798.197908   470.485294
                    6           40   2017  -193.611296   470.485294
...                            ...    ...          ...          ...
10      96          7          164  12102  3060.260216  2822.911765
                    8          164  12102  3134.273658  2822.911765
                    9          164  12102  3044.806001  2822.911765
                    10         164  12102  3098.952657  2822.911765
                    11         164  12102  2898.262451  2822.911765
                    12         164  12102  3050.134242  2822.911765
        97          1          164  12102  8073.877095  2822.911765
                    2          164  12102  7424.482365  2822.911765
                    3          164  12102  9119.044687  2822.911765
                    4          164  12102  7522.590331  2822.911765
                    5          164  12102  7379.530109  2822.911765
                    6          164  12102  8645.688944  2822.911765
                    7          164  12102  8752.329033  2822.911765
                    8          164  12102  8454.772732  2822.911765
                    9          164  12102  7912.554906  2822.911765
                    10         164  12102  8071.404733  2822.911765
                    11         164  12102  8315.121657  2822.911765
                    12         164  12102  8296.663202  2822.911765
        98          1           53   2017     0.000000   470.485294
                    2           53   2017     0.000000   470.485294
                    3           53   2017     0.000000   470.485294
                    4           53   2017     0.000000   470.485294
                    5           53   2017     0.000000   470.485294
                    6           53   2017     0.000000   470.485294
                    7           53   2017     0.000000   470.485294
                    8           53   2017     0.000000   470.485294
               

In [234]:
all_val.y_hat.unique()

array([470.48529422])